# Introduction 

This notebook deals with the prepocessing of the audio data in a MP3 format. It loads the MP3 files in full length and splits the signal in segments of equal length, such that the duration of every segmant is the same for every level of the model. 

# Imports and Gloabl Variables

* The library os is used get and crate the paths for the data. 
* The library numpy is used to create and load the training data and to execute mathmatical operations. 
* The library librosa is used to process audio data

In [ ]:
import os
import librosa
import numpy as np

In [ ]:
FINISH_SAMPLE_RATE = 22050
FILES_DIR = "C:/Masterarbeit/music_mp3" # the folder with the mp3 files
FILES_DIR_SAVE = "C:/Masterarbeit/music_npy" # the folder to save the .npy data
MONO = True  # if the data should be load mono
DURATION = 262144/FINISH_SAMPLE_RATE  # in seconds; 262144 is a power of 2 and the number of samples of the final level

# Generate Training Data

The code contains a loop to repeadely load the MP3 data with different sample rates. It starts with $22050/2^8$, which is slightly more than $86$. Every MP3 file is loaded with the respective sampling rate in full length starting at second five. The code then calculates how many samples have to lay in one segment, such that the defined duration is reached. The segments are then saved in the defined folder with a unique name. 

In [ ]:
def Loader(file_path, sample_rate, mono=True):
    '''
    converts a MP3 file to numeric wave data
    returns an numpy array
    
    file_path: path to the mp3 file to load
    sample_rate: sample_rate with which the mp3 should be loaded
    mono: if it should be loaded as mono
    '''
    signal = librosa.load(file_path,
                          sr=sample_rate,
                          offset = 5,
                          mono=mono)[0]
    return signal


def split_signal(signal,sr):
    '''
    takes in a signal and the sample_rate of the signal
    splits the signal into segments with the globally defined DURATION
    returns an two deminseional array with the split signal
    '''
    i = 0 # counter to define the end position of the array for the split signal
    j = 0 # counter to define the starting position of the array for the split signal
    number_samples_in_segment = int(DURATION * sr) # calculates the number of samples in one segment
    number_splits = (len(signal))//number_samples_in_segment # calcutes the number of segments
    splitted_signal = []  # instantiate a list to save the segments
    for _ in range(number_splits): # loop to extract and seve the segments
        i = i + number_samples_in_segment # define ending position
        if len(signal[j:i]) == number_samples_in_segment: # check if the number of samples is as expected
            splitted_signal.append(signal[j:i]) # save the segment
        j = j + number_samples_in_segment # define starting position
    return np.array(splitted_signal)


signals = [] # 
for i in range(0,9):  # loop to the number of growing iterations 
    sr = 22050/2**i # define the sampling rate for each iteration
    for root, _, files in os.walk(FILES_DIR): # loop over all subfolders 
        for file in files: # loop over all files in a subfolder
            file_path = os.path.join(root, file) # create the full path to a file
            try: signal = Loader(file_path, sr, DURATION, MONO) # try to load the file 
            except: print(f"Folgendes file konnte nicht geladen werden: {file}")
            else:
                splitted_signal = split_signal(signal,sr) # split the signal into segments of equal length
                j = 0 # counter to add to the name of the file
                for signal in splitted_signal: # loop through all segments
                    j = j + 1
                    filename = os.path.join(FILES_DIR_SAVE,f"sr_{int(sr)}", file) # create the new filename
                    np.save(f"{filename}_{j}",np.array(signal)) # save the segment as .npy file


The filenames of one of the subfolders with the training data for one level of the model are extracted and appended to one list and then saved as a numpy array. This is used later by the Data Generator to load the data in batches.

In [ ]:
# create file with all the filenames
filenames=[] # instantiate list for the filenames
FILE_NAMES_FOLDER = "C:/Masterarbeit/music_npy/sr_86" # define the folder from which the filenames should be extraced 
for root,_,files in os.walk(FILE_NAMES_FOLDER): # loop over all subfolders
    for file in files: # loop over all files
        filenames.append(file) # add the filename to the list
filename = os.path.join("C:/Masterarbeit", "filenames") # define the name of the new file
np.save(f"{filename}.npy",filenames) # save the list of all filenames as .npy file